# YOLOv11 Training with Roboflow Dataset

This notebook demonstrates how to train a YOLOv11 model using a dataset from Roboflow. It includes:
- Automatic GPU/CPU detection
- Configurable training parameters
- Training visualization and analysis

## Step 1: Install Dependencies
First, we'll install the required packages.

In [2]:
# For Training
!pip install ultralytics roboflow 

# For Storage
!pip install minio

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Step 2: Import Libraries
Import all necessary libraries for training and analysis.

In [5]:
# Common
import  os

# For Dataset manipulation
import yaml
from roboflow import Roboflow

# For training
import torch
from ultralytics import YOLO

# For Storage
from minio import Minio
from minio.error import S3Error

## Step 3: Download Dataset from Roboflow
Connect to Roboflow and download the dataset. Make sure to use your own API key and project details.

**Remember to replace the placeholders with your values**.

In [ ]:
rf = Roboflow(api_key="xxxxxxxxxxxxxxxxx")  # Replace with your API key
project = rf.workspace("yyyyyyyyyyyyyy").project("zzzzzzzzzzzzzzzzzzz") # Replace with your workspace and project names

version = project.version(1111111111111111111111111111) # Replace with your version number
dataset = version.download("yolov11")

You'll need to explicitly specify the paths to each data split (training, validation, and test) in your configuration. This ensures YOLO can correctly locate and utilize your dataset files.

This is done in the `data.yaml` file. If you open that file you will see these paths that you need to update:

```
train: ../train/images
val: ../valid/images
test: ../test/images
```

In [ ]:
print(f"Dataset downloaded to: {dataset.location}")

dataset_yaml_path = f"{dataset.location}/data.yaml"

with open(dataset_yaml_path, "r") as file:
    data_config = yaml.safe_load(file)

data_config["train"] = f"{dataset.location}/train/images"
data_config["val"] = f"{dataset.location}/valid/images"
data_config["test"] = f"{dataset.location}/test/images"

with open(dataset_yaml_path, "w") as file:
    yaml.safe_dump(data_config, file)

## Step 4: Configure Hyperparameters
Set up GPU/CPU detection (code automatically detects and use GPU if available).

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device} ({'GPU' if device.type == 'cuda' else 'CPU'})")

Using device: cuda:0 (GPU)


Define all training parameters in a single configuration dictionary.

In [ ]:

CONFIG = {
    'name': 'yolo_hardhat',
    'model': 'yolo11m.pt',  # Model size options: n, s, m, l, x
    'data': dataset.location + "/data.yaml",
    'epochs': 50,
    'batch': 16 if device != 'cpu' else 4,  # Adjust batch size based on device
    'imgsz': 640,
    'patience': 15,
    'device': device,
    
    # Optimizer settings
    'optimizer': 'SGD',
    'lr0': 0.001,
    'lrf': 0.005,
    'momentum': 0.9,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'warmup_bias_lr': 0.01,
    'warmup_momentum': 0.8,
    'amp': False,
    
    # Data augmentation settings
    'augment': True,
    'hsv_h': 0.015,  # HSV-Hue augmentation
    'hsv_s': 0.7,    # HSV-Saturation augmentation
    'hsv_v': 0.4,    # HSV-Value augmentation
    'degrees': 10,    # Image rotation (+/- deg)
    'translate': 0.1, # Image translation
    'scale': 0.3,    # Image scale
    'shear': 0.0,    # Image shear
    'perspective': 0.0,  # Image perspective
    'flipud': 0.1,   # Flip up-down
    'fliplr': 0.1,   # Flip left-right
    'mosaic': 1.0,   # Mosaic augmentation
    'mixup': 0.0,    # Mixup augmentation
}

# Configure PyTorch for GPU memory allocation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

## Step 5: Load Model
Initialize the YOLO model.

In [ ]:
model = YOLO(CONFIG['model'])

## Step 6: Start Training

Begin the training process. By default, the `train` method handles both "training" and "validation" sets. 

In [ ]:
results_train = model.train(
    name=CONFIG['name'],
    data=CONFIG['data'],
    epochs=CONFIG['epochs'],
    batch=CONFIG['batch'],
    imgsz=CONFIG['imgsz'],
    patience=CONFIG['patience'],
    device=CONFIG['device'],
    verbose=True,
    
    # Optimizer parameters
    optimizer=CONFIG['optimizer'],
    lr0=CONFIG['lr0'],
    lrf=CONFIG['lrf'],
    momentum=CONFIG['momentum'],
    weight_decay=CONFIG['weight_decay'],
    warmup_epochs=CONFIG['warmup_epochs'],
    warmup_bias_lr=CONFIG['warmup_bias_lr'],
    warmup_momentum=CONFIG['warmup_momentum'],
    amp=CONFIG['amp'],
    
    # Augmentation parameters
    augment=CONFIG['augment'],
    hsv_h=CONFIG['hsv_h'],
    hsv_s=CONFIG['hsv_s'],
    hsv_v=CONFIG['hsv_v'],
    degrees=CONFIG['degrees'],
    translate=CONFIG['translate'],
    scale=CONFIG['scale'],
    shear=CONFIG['shear'],
    perspective=CONFIG['perspective'],
    flipud=CONFIG['flipud'],
    fliplr=CONFIG['fliplr'],
    mosaic=CONFIG['mosaic'],
    mixup=CONFIG['mixup'],
)

## Step 7: Evaluate Model

 Evaluate the model on the test set.

In [ ]:
results_test = model.val(data=CONFIG['data'], split='test', device=CONFIG['device'], imgsz=CONFIG['imgsz'])

print("Test Results:", results_test)

## Step 7: (Optional) Model Export

Export the trained YOLO model to ONNX format for deployment.

In [ ]:
model.export(format='onnx', imgsz=CONFIG['imgsz'])

## Step 8: Store the Model

Save the trained model to the Object Storage system configured in your Workbench connection. 

Start by getting the credentials and configuring variables for accessing Object Storage.

In [ ]:
AWS_S3_ENDPOINT = os.getenv("AWS_S3_ENDPOINT")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")

Define the S3 client.

In [ ]:
client = Minio(
    AWS_S3_ENDPOINT,
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    secure=False
)

Select files to be uploaded (files generated while training and validating the model)

In [ ]:
model_path_train = results_train.save_dir
files_train = [file for file in model_path_train.rglob('*') if file.is_file()]

model_path_test = results_test.save_dir
files_test = [file for file in model_path_test.rglob('*') if file.is_file()]

Upload the files.

In [ ]:
directory_name= os.path.basename(model_path_train)

for file_path_train in files_train:
    try:
        client.fput_object(AWS_S3_BUCKET, directory_name + "/train-val/" + os.path.basename(file_path_train), file_path_train)
        print(f"'{os.path.basename(file_path_train)}' is successfully uploaded as object to bucket '{AWS_S3_BUCKET}'.")
    except S3Error as e:
        print("Error occurred: ", e)
        
for file_path_test in files_test:
    try:
        client.fput_object(AWS_S3_BUCKET, directory_name + "/test/" + os.path.basename(file_path_test), file_path_test)
        print(f"'{os.path.basename(file_path)}' is successfully uploaded as object to bucket '{AWS_S3_BUCKET}'.")
    except S3Error as e:
        print("Error occurred: ", e)